In [1]:
# UFLP - Uncapacitated Facilitated Location Problem

# Imamo I korisnika i J resursa Pravimo bolnice u gradu od 0 
# i hocemo najbolje lokacije za bolnice tako da budu sto blize i sto jeftinije

# Cij - cena pridruzivanja korisnika i resursu j 
# fij - cena uspostavljanja resursa j


# 10 korisnika
# 5 bolnica
# c
# f

# solution = [True, False, False, True, False]

## Local Search

In [1]:
import random

In [3]:
def isFeasible(solution):
    for resource in solution:
        if resource:
            return True
    return False

In [5]:
def initialize(numResources):
    solution = [random.random() < 0.25 for _ in range(numResources)]
    if not isFeasible(solution):
        chosenResource = random.randrange(0, numResources)
        solution[chosenResource] = True
    return solution

In [16]:
def calcSolutionValue(solution, cost, fixedCost):
    
    value = 0
    numUsers = len(cost)
    numResources = len(fixedCost)
    
    used = [False for _ in range(numResources)]
    
    for i in range(numUsers):
        bestCost = float('inf')
        rUsed = -1
        for j in range(numResources):
            if solution[j] and cost[i][j] < bestCost:
                rUsed = j
                bestCost = cost[i][j]
        value += bestCost
        used[rUsed] = True
        
    for i in range(numResources):
        if used[i]:
            value += fixedCost[i]
            
    return value

In [8]:
# probamo drugo resenje - samo flipujemo jedan True/False
def invert(solution):
    randomResource = random.randrange(0, len(solution))
    solution[randomResource] = not solution[randomResource]
    
    if not isFeasible(solution):
        solution[randomResource] = not solution[randomResource]
        return -1, solution
    return randomResource, solution

In [13]:
def localSearch(cost, fixedCost, iters):
    # initialize solution, bestValue, current value
    solution = initialize(len(fixedCost))
    currentValue = calcSolutionValue(solution, cost, fixedCost)
    bestValue = currentValue
    # for every iter
    for i in range(iters):
        # invert solution
        success, solution = invert(solution)
        
        if success < 0:
            continue       
        # calculate new solution value
        newValue = calcSolutionValue(solution, cost, fixedCost)
        # if it's better than current value, update current
        if newValue < currentValue:
            currentValue = newValue
            # if it's better than best value, update best
            if newValue < bestValue:
                bestValue = newValue
            # else: return to previous solution
            else:
                solution[success] = not solution[success]
        
    # return best value
    return bestValue

In [14]:
def readInput(filename):
    with open(filename, 'r') as f:
        numUsers, numResources = [int(x) for x in f.readline().split()]
        cost = [[int(x) for x in f.readline().split()] for _ in range(numUsers)]
        fixedCost = [int(x) for x in f.readline().split()]
        return cost, fixedCost

In [23]:
cost, fixedCost = readInput("uflp1.txt")
localSearch(cost, fixedCost, 1000)

34

## Simulirano Kaljenje

Uzimamo u obzir lokalne ekstremume: Sa nekom verovatnocom cemo ponekad uzeti losije resenje namerno, kako bismo ispitali potencijalan bolji rezultat u drugoj okolini

In [44]:
from copy import deepcopy

In [97]:
def simulatedAnnealing(cost, fixedCost, iters):
    solution = initialize(len(fixedCost))
    currValue = calcSolutionValue(solution, cost, fixedCost)
    bestValue = currValue
    bestSolution = deepcopy(solution)
    
    for i in range(iters):
        success, newSolution = invert(solution)
        if success < 0:
            continue
            
        newValue = calcSolutionValue(newSolution, cost, fixedCost)
        
        if newValue < currValue:
            currValue = newValue
            solution = deepcopy(newSolution)
            
            if newValue < bestValue:
                bestValue = newValue
                bestSolution = deepcopy(newSolution)
        
        else: 
            p = 1.0/(i+1.0)
            q = random.uniform(0,1)
            
            if p > q:
                currValue = newValue
                
    return bestValue, bestSolution

In [96]:
simulatedAnnealing(cost, fixedCost, 100)

(34, [True, False, False])

## Metoda promenljivih okolina

Vise okolina - pokusavamo da sirimo raspon u kojem razmatramo sledecu opciju za solution. 

Mi radimo REDUKOVANU METODU

In [84]:
# Pseudokod: 

# solution = initialize()
# currValue = calcSolutionValue(solution)
# bestValue = currValue

# while not stop_condition():
#     k = 0
#     while k <= max_neighborhoods:
#         newSolution = getNeighbor(solution, k)
#         newValue = calcSolutionValue(newSolution)
        
#         if newValue < currValue:
#             currValue = newValue
#             solution = newSolution
            
#         else:
#             k += 1
# return bestSolution

In [95]:
def getNeighbor(solution, k):
    indices = range(len(solution))
    toInvert = random.sample(indices, k)
    
    for index in toInvert:
        solution[index] = not solution[index]
        
    if isFeasible(solution):
        return True, toInvert
    else:
        return False, toInvert

In [90]:
def restore(solution, invertedIndices):
    for resource in invertedIndices:
        solution[resource] = not solution[resource] 

In [100]:
def RVNS(cost, fixedCost, iters, maxK):
    solution = initialize(len(fixedCost))
    currValue = calcSolutionValue(solution, cost, fixedCost)
    bestValue = currValue
    
    for i in range(iters):
        k = 0
        while k <= maxK:
            success, invertedIndices = getNeighbor(solution, k)
            if not success:
                #vrati se na originalni solution
                restore(solution, invertedIndices)
                continue        
                
            newValue = calcSolutionValue(solution, cost, fixedCost)
            if newValue < currValue:
                currValue = newValue
                
                if newValue < bestValue:
                    bestValue = newValue
                    
                break
            else: 
                k += 1
                restore(solution, invertedIndices)
                
    return bestValue

In [101]:
RVNS(cost, fixedCost, 10, 2)

34